In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as md

In [ ]:
init_data = pd.read_csv("GOOG.csv")

In [ ]:
print(init_data.info())

In [ ]:
init_data.head()

In [ ]:
data = init_data.drop(["Adj Close"], axis=1).copy()

In [ ]:
data.head()

In [ ]:
data["Date"] = pd.to_datetime(data["Date"])

In [ ]:
plt.figure(figsize=(14, 8))
plt.plot('High', data=init_data)
plt.plot('Low', data=init_data,)
plt.xlabel("HIGH_LOW")
plt.ylabel("US_Dollar")
plt.legend()
plt.Normalize(vmin=0, vmax=100, clip=True)
plt.grid(which='both')

In [ ]:
plt.figure(figsize=(14,8))
plt.plot(data["Volume"])
plt.ylabel("Volume")
plt.xlabel("Days")
plt.legend("V",loc="upper left")

In [ ]:
data.isna().sum()

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split

In [ ]:
train_cols = data.columns[1:]
print(train_cols)

In [ ]:
min_max = MinMaxScaler()
norm_data = min_max.fit_transform(data[train_cols])

In [ ]:
X_train, X_test = train_test_split(norm_data, test_size=0.2, train_size=0.8, shuffle=False)

In [ ]:
def dataseries(mat, y_col, time_stamp):
    
    dim0 = mat.shape[0] - time_stamp
    dim1 = mat.shape[1]
    
    x = np.zeros((dim0,time_stamp,dim1))
    y = np.zeros((dim0))
     
    for itr in range(dim0):
        x[itr] = mat[itr:time_stamp+itr]
        y[itr] = mat[time_stamp, y_col]
        
    return x,y

In [ ]:
def trimseries(mat, batch):
    
    no_of_row_remined = mat.shape[0]%batch
    
    if no_of_row_remined != 0 :
        mat = mat[:-no_of_row_remined]
        
    return mat

In [ ]:
y_col = 3
time_stamp = 60
batch_size = 20
learning_rate = 0.001
OUTPUT_PATH = "C:\\Users\\varun\\Documents\\Pro\jects\\Stock-Market-Prediction"
epochs = 200

In [ ]:
x_t, y_t = dataseries(X_train, y_col, time_stamp)
x_t = trimseries(x_t, batch_size)
y_t = trimseries(y_t, batch_size)
x_test, y_test = dataseries(X_test, y_col, time_stamp)
x_val, x_test_t  = np.split(trimseries(x_test, batch_size),2)
y_val, y_test_t  = np.split(trimseries(y_test, batch_size),2)

In [ ]:
import tensorflow as tf
import tensorflow.keras as k
import os
import sys
sys.executable

In [ ]:
#model.reset_states()

In [ ]:
x_t.shape[2]

In [ ]:
def Model(batch_size, learning_rate):
    
    model = k.models.Sequential()
    
    model.add(k.layers.LSTM(128, batch_input_shape=(batch_size, 60, 5), dropout=0.0, recurrent_dropout=0.0, stateful=True,     kernel_initializer='random_uniform'))
    model.add(k.layers.Dropout(0.2))
    model.add(k.layers.Dense(32,activation='relu'))
    model.add(k.layers.Dense(1,activation='sigmoid'))
    
    optimizer = k.optimizers.RMSprop(learning_rate = learning_rate)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    return model

In [ ]:
def fit_model(model, OUTPUT_PATH, epochs, X_val, batch_size, y_val):
    
    csv_logger = k.callbacks.CSVLogger(os.path.join(OUTPUT_PATH, 'LSTM_model' + '.log'), append=True)
    
    history = model.fit(x_t, y_t, epochs=epochs, verbose=2, batch_size=batch_size,
                    shuffle=False, validation_data=(trimseries(x_val, batch_size),
                    trimseries(y_val, batch_size)))
    return history

In [ ]:
def predict(X_test, y_test, batch_size):
    
    y_pred = model.predict(trimseries(x_test_t, batch_size),batch_size=batch_size)
    y_pred  = y_pred.flatten()
    y_test_t = trimseries(y_test_t, batch_size)
    erorr = k.metrics.mean_squared_error(y_test_t,y_pred)
    
    return error

In [ ]:
#y_pred_org = (y_pred * min_max.data_range_[3]) + min_max.data_min_[3]
#y_test_t_org = (y_test_t * min_max.data_range_[3]) + min_max.data_min_[3]

In [ ]:
def plot_diff(history):
    
    ax = plt.figure(figsize=(12,8))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.xlabel("loss")
    plt.ylabel("Val_loss")
    #plt.legend(labels)
    print(ax)
    pass

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = dict(epochs = [10, 20, 30], 
                  batch_size = [20, 60, 90], 
                  learning_rate = [0.1, 0.01, 0.001])

In [ ]:
model_grid = k.wrappers.scikit_learn.KerasRegressor(build_fn=Model)

In [ ]:
grid = GridSearchCV(estimator=model_grid, param_grid=parameters, cv=3)
grid_result = grid.fit(x_t, y_t)

In [ ]:
#model = Model(batch_size, time_stamp, x_t, learning_rate)

In [ ]:
#fit_model(model, OUTPUT_PATH, epochs, x_val, batch_size, y_val)

In [ ]:
grid_result.best_result_